In [1]:
!pip install transformers==4.19.2
!pip install rouge_metric
!pip install nltk==3.6.5
!pip install sentencepiece
from google.colab import drive
drive.mount('./mydata')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at ./mydata; to attempt to forcibly remount, call drive.mount("./mydata", force_remount=True).


In [2]:
import sys
sys.path.append('./mydata/MyDrive/CSNLP_Project/Bert_model_COQA')
sys.path.append('./mydata/MyDrive/CSNLP_Project/T5_model_COQAR')
sys.path.append('./mydata/MyDrive/CSNLP_Project/T5_model_COQAR/rewriting')

In [3]:
import collections
import glob
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
from transformers import (AdamW, AutoConfig, AutoTokenizer, get_linear_schedule_with_warmup, BertTokenizer, BertModel, BertConfig)
from data.processors.coqa import Extract_Features, Processor, Result
from data.processors.evaluate import CoQAEvaluator, parse_args
from data.processors.Bert_model import BertBaseUncasedModel, load_dataset, Write_predictions

#from transformers import BertModel, BertPreTrainedModel

import csv
import numpy as np

import evaluation
import argparse
import qrdatasets
import models
from utils import *
import random
import t5small
import t5base
import nltk

import config
import json

# CoQA dataset file
train_file="coqa-train-v1.0.json"
predict_file="coqa-dev-v1.0.json"
cur_path = os.getcwd()
output_directory = cur_path + "/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models"
input_dir = cur_path + "/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data"
# can use either BERT base or BERT large
pretrained_model="bert-base-uncased"
# pretrained_model="bert-large-uncased"
# it's better to fine-tune Bert-base for 4 epoches than only one
epochs = 4
evaluation_batch_size=1
train_batch_size=2

In [4]:
### Load two fine-tuned models, but with diffetent load methods

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load bert
# bert_path = './mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_from_original_Surya_epoch4'
# bert_model = BertBaseUncasedModel.from_pretrained(bert_path) 
# bert_tokenizer = BertTokenizer.from_pretrained(bert_path, do_lower_case=True)
# bert_model.to(device)

# load t5
# t5_path = './mydata/MyDrive/CSNLP_Project/T5_model_COQAR/trained_models/t5_small_with_story_batch16_hist_3_mixed/epoch10'
t5_path = './mydata/MyDrive/CSNLP_Project/T5_model_COQAR/trained_models/t5_base_with_story_batch4_hist_20/epoch6.zip'

t5_model = torch.load(t5_path)
# t5_input_tokenizer = t5small.get_input_tokenizer()
# t5_output_tokenizer = t5small.get_output_tokenizer()
t5_input_tokenizer = t5base.get_input_tokenizer()
t5_output_tokenizer = t5base.get_output_tokenizer()
t5_model.to(device)

pass

In [4]:
# default parameters for T5
hparams = {
    'epochs' : 3,
    'learning_rate' : 0.00005,
    'batch_size' : 16,
    'weight_decay' : 0.0,
    'history_size' : 3,
    'dropout_rate' : 0.1,
    'include_story' : True,
    'model_size' : 'small'
}
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# get rewritten answers from T5

def generate_new_dataset(input_dir,file_type='train',t5_type='base',t5_model=t5_model,method='append'):
### open the COQA dataset
    if file_type == 'train':
        # training dataset
        with open(os.path.join(input_dir, train_file), "r", encoding="utf-8") as reader:
            input_data = json.load(reader)
    elif file_type == 'dev':
        # dev dataset
        with open(os.path.join(input_dir, predict_file), "r", encoding="utf-8") as reader:
            input_data = json.load(reader)
    else:
        raise Exception("must specify a file type: train/dev")
        pass

    # create empty dictionary
    t5_data = {'input':[], 'references':[], 'context':[]}

    # create dataset for T5
    for input_d in input_data["data"]:
        questions = []
        assert len(input_d['questions']) == len(input_d['answers'])
        for id_q in range(len(input_d['questions'])):
            if id_q == 0:
                questions.append(input_d['questions'][id_q]['input_text'])
            else:
                questions.append(input_d['answers'][id_q-1]['input_text'])
                questions.append(input_d['questions'][id_q]['input_text'])
            t5_data['input'].append(questions.copy())
            t5_data['references'].append([''])
            t5_data['context'].append(input_d['story'])

    # T5 make dataset
    if t5_type == 'base':
        dataset = t5base.make_dataset(t5_data, hparams, cuda = True)
    elif t5_type == 'small':
        dataset = t5base.make_dataset(t5_data, hparams, cuda = True)
    else:
        raise Exception("must specify a t5 model type: base/small")
        pass

    # generate rewritten questions
    loader = DataLoader(dataset=dataset, batch_size=hparams['batch_size'])
    t5_model.cuda()
    t5_model.train(False)
    rewritten_qs = []
    for dic in loader:
        output = t5_model.generate(input_ids = dic['input_ids'], attention_mask = dic['attention_mask'])
        pred = t5_output_tokenizer.batch_decode(output, skip_special_tokens = True)
        rewritten_qs += pred

    ### Create new training/dev dataset json file for Bert
    # append or replace with the generated question 
    count = 0
    for story_id in range(len(input_data["data"])):
        for question_id in range(len(input_data["data"][story_id]['questions'])):
            if method == 'append':
                # append the rewritten question
                input_data["data"][story_id]['questions'][question_id]['input_text'] += (' '+rewritten_qs[count])
            elif method == 'replace':
                # replace with the rewritten question
                input_data["data"][story_id]['questions'][question_id]['input_text'] = rewritten_qs[count]
            else:
                raise Exception("must specify a method: append/replace")
                pass
            count += 1

    # save as new dataset json file
    file_name = 'coqa-{}-v1.0-{}_with_T5.json'.format(file_type,method)
    # with open(os.path.join( input_dir, 'coqa-dev-v1.0-with_T5.json'), 'w', encoding="utf-8") as outfile:
    with open(os.path.join(input_dir, file_name), 'w', encoding="utf-8") as outfile:
        json.dump(input_data, outfile)


generate_new_dataset(input_dir,file_type='dev',t5_type='base',t5_model=t5_model,method='replace')

## Prediction

predict on dev dataset

In [5]:
### Load two fine-tuned models, but with diffetent load methods

model_parameter_directory = [ f.path for f in os.scandir(output_directory) if f.is_dir() ]

method = '_replace'

# use catch file name
cache_file_name = 'bert-base-uncased_dev_with_T5{}'.format(method)
# use the predict file name
predict_file_name = 'coqa-dev-v1.0-{}_with_T5.json'.format(method[1:])
# # reset the catch file name
# cache_file_name = None
# # reset the predict file name
# predict_file_name = None

model_parameter_directory

['/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_from_original_Surya_epoch4',
 '/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_with_T5_rewritten_epoch4',
 '/content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/Bert_models/Bert_with_T5_rewritten_epoch4_replace']

In [6]:
# for m in model_parameter_directory:
m = model_parameter_directory[2]
variant_name = m.split('/')[-1]
print(variant_name)
# m = m + '/pytorch_model_2.bin'
model = BertBaseUncasedModel.from_pretrained(m) 
tokenizer = BertTokenizer.from_pretrained(m, do_lower_case=True)
model.to(device)
Write_predictions(model, tokenizer, device, variant_name, input_dir=input_dir, output_directory=output_directory, cache_file_name=cache_file_name, predict_file_name=predict_file_name, method=method)

Bert_with_T5_rewritten_epoch4_replace
Loading cache /content/mydata/MyDrive/CSNLP_Project/Bert_model_COQA/data/bert-base-uncased_dev_with_T5_replace


Writing preditions: 100%|██████████| 7983/7983 [00:28<00:00, 284.16it/s]


## Evaluation

In [ ]:
evaluator = CoQAEvaluator(input_dir+'/'+predict_file_name)

variant_name = 'Bert_with_T5_rewritten_epoch4_replace'

pre_file_bert = output_directory+'/'+variant_name+'/'+'predictions{}.json'.format(method)

# evaluate
with open(pre_file_bert) as f:
    pred_data = CoQAEvaluator.preds_to_dict(pre_file_bert)

# write evaluate results
with open(output_directory+'/'+variant_name+'/'+'evaluation{}.json'.format(method), 'w') as f:
    json.dump(evaluator.model_performance(pred_data), f, indent=2)

# show
print(json.dumps(evaluator.model_performance(pred_data), indent=2))